In [51]:
import os
from pyprojroot import here
os.environ["HF_HOME"] = str(here("cache/huggingface"))
import lightning as L
from importlib import reload
import sys
sys.path.append(str(here()))
from models import model_definitions as MD
import torch
import pandas as pd
reload(MD)
torch.set_float32_matmul_precision('medium')
data_module_3 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_3.parquet")))
data_module_7 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_7.parquet")))
data_module_28 = MD.GoEmotionsDataModule(pd.read_parquet(here("data/goemotions_28.parquet")))
metrics = MD.MetricsCallback()

def make_trainer():
  return L.Trainer(
    max_epochs=15,
    max_time={"minutes": 10},
    callbacks=[
      # metrics,
    ],
    deterministic=True,
    default_root_dir=here("cache/lightning"),
    enable_checkpointing=True,
    fast_dev_run=False,
    accelerator="auto", 
  )

trainer = make_trainer()
trainer

Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


# Distil Bert finetuned (cross entropy loss)

## 3 classes

In [52]:
reload(MD)
distil_3 = MD.DistilBertFinetune(n_emotions=3)
if False:    
  trainer.fit(
    distil_3,
    datamodule=data_module_3,
  )
else:
  distil_3 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_3_14epchs.ckpt"),
    n_emotions=3,
  )
out = trainer.test(
  distil_3,
  datamodule=data_module_3,
)
del distil_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\Plancha\emotion-recognition\.pixi\envs\gpu\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:425: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_work

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4729805886745453
    test_expectedNDCG       0.6360554099082947
    test_f1_interest        0.5236578583717346
      test_f1_stand         0.5838572978973389
        test_nDGC           0.9337419867515564
        test_rmse           0.25479617714881897
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4729805886745453,
  'test_f1_stand': 0.5838572978973389,
  'test_f1_interest': 0.5236578583717346,
  'test_rmse': 0.25479617714881897,
  'test_nDGC': 0.9337419867515564,
  'test_expectedNDCG': 0.6360554099082947}]

## 7 classes

In [53]:
reload(MD)
distil_7 = MD.DistilBertFinetune(n_emotions=7)
if False:    
  trainer.fit(
    distil_7,
    data_module=data_module_7,
  )
else:
  distil_7 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_7_14epchs.ckpt"),
    n_emotions=7,
  )
out = trainer.test(
  distil_7,
  datamodule=data_module_7
)
del distil_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.2803645133972168
    test_expectedNDCG       0.41903358697891235
    test_f1_interest        0.2714906632900238
      test_f1_stand         0.23877769708633423
        test_nDGC           0.8969202041625977
        test_rmse           0.18574778735637665
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.2803645133972168,
  'test_f1_stand': 0.23877769708633423,
  'test_f1_interest': 0.2714906632900238,
  'test_rmse': 0.18574778735637665,
  'test_nDGC': 0.8969202041625977,
  'test_expectedNDCG': 0.41903358697891235}]

## 28 classes

In [54]:
reload(MD)
distil_28 = MD.DistilBertFinetune(n_emotions=28)
trainer = make_trainer()
if False:    
  trainer.fit(
    distil_28,
    datamodule=data_module_28,
  )
else:
  distil_28 = MD.DistilBertFinetune.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_finetuned_28_11epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_28,
  datamodule=data_module_28,
)
del distil_28
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.11658827215433121
    test_expectedNDCG       0.2731611430644989
    test_f1_interest        0.09542364627122879
      test_f1_stand         0.09084004908800125
        test_nDGC           0.8161150217056274
        test_rmse           0.11110500991344452
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.11658827215433121,
  'test_f1_stand': 0.09084004908800125,
  'test_f1_interest': 0.09542364627122879,
  'test_rmse': 0.11110500991344452,
  'test_nDGC': 0.8161150217056274,
  'test_expectedNDCG': 0.2731611430644989}]

# Distil Bert finetuned (DCG loss)

## 3 classes

In [55]:
reload(MD) # DistilBertFinetuneOnDCG
distil_dcg_3 = MD.DistilBertFinetuneOnDCG(n_emotions=3)
trainer = make_trainer()
if False:    
  trainer.fit(
    distil_dcg_3,
    datamodule=data_module_3,
  )
else:
  distil_dcg_3 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_3_10epchs.ckpt"),
    n_emotions=3,
  ) 
out = trainer.test(
  distil_dcg_3,
  datamodule=data_module_3,
)
del distil_dcg_3
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.5900861024856567
    test_expectedNDCG       0.6335099339485168
    test_f1_interest        0.4503559172153473
      test_f1_stand         0.7225136756896973
        test_nDGC           0.9327593445777893
        test_rmse           0.33047133684158325
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.5900861024856567,
  'test_f1_stand': 0.7225136756896973,
  'test_f1_interest': 0.4503559172153473,
  'test_rmse': 0.33047133684158325,
  'test_nDGC': 0.9327593445777893,
  'test_expectedNDCG': 0.6335099339485168}]

## 7 classes

In [56]:
reload(MD)
distil_dcg_7 = MD.DistilBertFinetuneOnDCG(n_emotions=7)
trainer = make_trainer()
if False:    
  trainer.fit(
    distil_dcg_7,
    datamodule=data_module_7,
  )
else:
  distil_dcg_7 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_7_10epchs.ckpt"),
    n_emotions=7,
  )
  
out = trainer.test(
  distil_dcg_7,
  datamodule=data_module_7,
)
del distil_dcg_7
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.42078444361686707
    test_expectedNDCG       0.4177568554878235
    test_f1_interest        0.18308162689208984
      test_f1_stand         0.39270398020744324
        test_nDGC           0.8886398077011108
        test_rmse           0.2857814431190491
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.42078444361686707,
  'test_f1_stand': 0.39270398020744324,
  'test_f1_interest': 0.18308162689208984,
  'test_rmse': 0.2857814431190491,
  'test_nDGC': 0.8886398077011108,
  'test_expectedNDCG': 0.4177568554878235}]

## 28 classes

In [57]:
reload(MD)
distil_dcg_28 = MD.DistilBertFinetuneOnDCG(n_emotions=28)
trainer = make_trainer()
if False:
  trainer.fit(
    distil_dcg_28,
    datamodule=data_module_28,
  )
else:
  distil_dcg_28 = MD.DistilBertFinetuneOnDCG.load_from_checkpoint(
    here("models/model_checkpoints/distil_bert_onDCG_28_10epchs.ckpt"),
    n_emotions=28,
  )
out = trainer.test(
  distil_dcg_28,
  datamodule=data_module_28,
)
out

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
LOCAL_RANK: 0 - CUDA

Testing: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   test_cross_entropy       0.4481115937232971
    test_expectedNDCG       0.2697453200817108
    test_f1_interest       0.052236463874578476
      test_f1_stand         0.3579895496368408
        test_nDGC           0.7873262763023376
        test_rmse            0.342868834733963
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_cross_entropy': 0.4481115937232971,
  'test_f1_stand': 0.3579895496368408,
  'test_f1_interest': 0.052236463874578476,
  'test_rmse': 0.342868834733963,
  'test_nDGC': 0.7873262763023376,
  'test_expectedNDCG': 0.2697453200817108}]

In [58]:
# test sample
trainer.predict(
  distil_dcg_28,
  ["I Love this"],
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: |          | 0/? [00:00<?, ?it/s]

[tensor([[0.0433, 0.6453, 0.9952, 0.3850, 0.0246, 0.2627, 0.0545, 0.0138, 0.9210,
          0.1429, 0.0670, 0.0282, 0.2872, 0.1308, 0.5913, 0.3643, 0.8826, 0.0120,
          0.6599, 0.0147, 0.0060, 0.0908, 0.0364, 0.0083, 0.1318, 0.5724, 0.0764,
          0.0056]])]

In [59]:
pd.read_parquet(here("data/goemotions_28.parquet")).columns

Index(['id', 'text', 'n_raters', 'emotions', 'author', 'subreddit', 'link_id',
       'parent_id', 'created_utc', 'emotion_sadness', 'emotion_unclear',
       'emotion_love', 'emotion_gratitude', 'emotion_disapproval',
       'emotion_amusement', 'emotion_disappointment', 'emotion_disgust',
       'emotion_admiration', 'emotion_realization', 'emotion_annoyance',
       'emotion_confusion', 'emotion_optimism', 'emotion_curiosity',
       'emotion_excitement', 'emotion_caring', 'emotion_joy',
       'emotion_remorse', 'emotion_approval', 'emotion_nervousness',
       'emotion_embarrassment', 'emotion_surprise', 'emotion_anger',
       'emotion_grief', 'emotion_pride', 'emotion_desire', 'emotion_relief',
       'emotion_fear'],
      dtype='object')